# Model Deployment

## Importing the required Libraries

In [1]:
import joblib
from pydantic import BaseModel, Field 
from fastapi import FastAPI, HTTPException
from typing import List
import os
from dotenv import load_dotenv
import uvicorn
from subprocess import Popen
import pandas as pd

# Loading the .env files
load_dotenv()

True

## Loading the model and the preprocessor

In [2]:
model_path = os.getenv('MODEL_PATH')
preprocessor_path = os.getenv('PREPROCESSOR_MODEL_PATH')

rf_classifier = joblib.load(model_path)
preprocessor = joblib.load(preprocessor_path)

## Creating a class for Data Preprocessing and Model Prediction


In [3]:
class ModelHandler:
    def __init__(self, preprocessor, model):
        self.preprocessor = preprocessor
        self.model = model
    
    def preprocess_data(self, new_data):
        df = pd.DataFrame(new_data)
        processed_data = self.preprocessor.transform(df)
        return processed_data
    
    def predict(self, new_data):
        processed_data = self.preprocess_data(new_data)
        predictions = self.model.predict(processed_data)
        return predictions

## FastAPI Setup

In [4]:
# Handler
handler = ModelHandler(preprocessor, rf_classifier)

# Fast API Setup
app = FastAPI()

class RequestedItem(BaseModel):
    Age: List[float]
    Number_of_Dependents: List[int] = Field(..., alias="Number of Dependents")
    Zip_Code: List[int] = Field(..., alias="Zip Code")
    Number_of_Referrals: List[int] = Field(..., alias="Number of Referrals")
    Tenure_in_Months: List[int] = Field(..., alias="Tenure in Months")
    Avg_Monthly_Long_Distance_Charges: List[float] = Field(..., alias="Avg Monthly Long Distance Charges")
    Avg_Monthly_GB_Download: List[float] = Field(..., alias="Avg Monthly GB Download")
    Monthly_Charge: List[float] = Field(..., alias="Monthly Charge")
    Total_Charges: List[float] = Field(..., alias="Total Charges")
    Total_Refunds: List[float] = Field(..., alias="Total Refunds")
    Total_Extra_Data_Charges: List[float] = Field(..., alias="Total Extra Data Charges")
    Total_Long_Distance_Charges: List[float] = Field(..., alias="Total Long Distance Charges")
    Total_Revenue: List[float] = Field(..., alias="Total Revenue")
    Satisfaction_Score: List[int] = Field(..., alias="Satisfaction Score")
    Churn_Score: List[int] = Field(..., alias="Churn Score")
    CLTV: List[int]
    Gender: List[str]
    Under_30: List[str] = Field(..., alias="Under 30")
    Senior_Citizen: List[str] = Field(..., alias="Senior Citizen")
    Married: List[str]
    Dependents: List[str]
    City: List[str]
    Phone_Service: List[str] = Field(..., alias="Phone Service")
    Multiple_Lines: List[str] = Field(..., alias="Multiple Lines")
    Internet_Service: List[str] = Field(..., alias="Internet Service")
    Online_Security: List[str] = Field(..., alias="Online Security")
    Online_Backup: List[str] = Field(..., alias="Online Backup")
    Device_Protection_Plan: List[str] = Field(..., alias="Device Protection Plan")
    Premium_Tech_Support: List[str] = Field(..., alias="Premium Tech Support")
    Streaming_TV: List[str] = Field(..., alias="Streaming TV")
    Streaming_Movies: List[str] = Field(..., alias="Streaming Movies")
    Streaming_Music: List[str] = Field(..., alias="Streaming Music")
    Unlimited_Data: List[str] = Field(..., alias="Unlimited Data")
    Contract: List[str]

    class Config:
        populate_by_name = True

@app.post('/predict/')
async def predict(request: List[RequestedItem]):
    try:
        data = [item.dict(by_alias=True, exclude_unset=True, exclude_defaults=True) for item in request]
        data = data[0]
        prediction = handler.predict(data)
        print(prediction)
        return {
            "Prediction": prediction.tolist()
        }
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    await server.serve()


INFO:     Started server process [10144]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


['No']
INFO:     127.0.0.1:60348 - "POST /predict/ HTTP/1.1" 200 OK


g:\ML Portfolio\end-to-end machine learning project\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


['No']
INFO:     127.0.0.1:60361 - "POST /predict/ HTTP/1.1" 200 OK


g:\ML Portfolio\end-to-end machine learning project\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [10144]
